## <center> Introduction </center>

I chose to implement the Apriori algorithm to gain a better understanding of the theory behind it and to further explore the data mining and association rule generating algorithms that drive our modern digital world. I chose not to use any pre-implemented functions from the many robust data science packages that exist for Python, as doing so would negate the purpose of this exercise.

### Input 

This program is written to read in a CSV file of itemsets. These itemsets can be actual transactions such as milk, beer, diapers, or they can be groups of numbers such as 1,2,3,4. The CSV must not have a header and there can only be one transaction per line, for example:\
*<center> 1,2,3,4 </center>*\
*<center> 1,2,4 </center>*\
*<center> 1,2,3,4,5,6 </center>*\
*<center> 1,2,5,4 </center>*

In [2]:
import csv

def ReadCSV(inputFile):
    C1 = []
    inputTextToList = []
    numberOfLines = 0

    with open(inputFile, "r") as f:
        csvFile = csv.reader(f)
        for line in csvFile:
            inputTextToList.append(line)
            for word in line:
                tempWordList = []
                tempWordList.append(word)
                if tempWordList not in C1:
                    C1.append(tempWordList)
            numberOfLines += 1 
    return inputTextToList, numberOfLines, C1

As the file is read in, a copy of the file is stored in a list of lists. At the same time, the number of lines in the file is counted, as this is important for calculating the support of items. Finally, the first candidate set is created by appending the current item to a list every time a unique item is encountered.

### Pruning

Once C1 is generated and the minimum support is formulated as a percent, the candidate set can be pruned of infrequent items.

In [3]:
def PruneInfrequentItemsets(CK, minimumSupport, inputTextToList):
    LK = []
    for itemSet in CK:
        if Support(itemSet, inputTextToList) >= minimumSupport:
            LK.append(itemSet)
    return LK

The pruning is very simple. With the candidate set (CK), minimum support, and a copy of the file supplied, all that needs to be done is to compare the support of each itemset in the candidate set to the minimum support. If the support for an itemset is greater than or equal to the provided support, then the itemset is appended to LK.

In [ ]:
def Support(itemSet, inputTextToList):
    numberOfOccurences = 0
    for line in inputTextToList:
        if all(e in line for e in itemSet):
            numberOfOccurences += 1
    return numberOfOccurences

Calculating support is equally simple. If every item in an itemset occurs in an itemset in the original file, then the number of occurrences (support) is increased by one for that itemset.

### Apriori

Once C1 and subsequently L1 have been generated, the main loop of the algorithm is ready to be run.

In [ ]:
LK = L1
frequentItemSets = []
K = 2
    
while LK:
    frequentItemSets.extend(LK)
    #generate possile combinations (candidate set)
    CK = utils.GenerateCandidateSet(LK, K)
    LK.clear()
    #prune the infrequent items save to LK
    LK = utils.PruneInfrequentItemsets(CK, minSupport, inputTextToList)
    K += 1

In this loop, the exact steps as described above occur in a similar order with one extra step, the generation of new candidate sets. First, LK is appended to a list of frequent itemsets. Next, a new candidate set is generated using the previous frequent itemsets. Then, LK is cleared so that the pruned CK can be saved and the process can start again, ending once there are no more itemsets that satisfy the minimum support.

In [ ]:
def GenerateCandidateSet(itemset, K):
    CK = []
    requiredNumberOfMatchingElements = (K - 2)
    for x in range(len(itemset)):
        for y in range(x + 1, len(itemset)):
            #if the first K - 2 elements are the same then make combination
            if MatchingFirstKElements(itemset[x],itemset[y], requiredNumberOfMatchingElements):
                #make a new list from joining x and y
                newList = itemset[x] + itemset[y]
                #remove duplicates
                newList = list(dict.fromkeys(newList))
                #append to CK
                CK.append(newList)
    return CK

When generating frequent itemsets from previously frequent itemsets, it is important to remember that two itemsets must have K - 2 matching first elements. For example, when generating C2, the itemsets [1] and [2] do not have to match; however, when generating C3, the itemsets [1, 2] and [1, 3] must match the first item to be combined.

To generate candidate sets, every itemset (i) needs to be compared with every next itemset in the list (i+1). For each itemset, the number of matching first items needs to be checked. If the items match, then the itemsets can be combined. First, the itemsets are appended together; from there, the duplicate items are removed. Finally, this new itemset is added to the new candidate set.

In [ ]:
def MatchingFirstKElements(list1, list2, K):
    for i in range(K):
        if list1[i] != list2[i]:
            return False
    return True

To match the first K - 2 items of two itemsets, the two itemsets must be iterated over for however large K is at that iteration of the loop. If at any point the two itemsets do not match, then the comparison can be stopped.

### Association Rules

Once the frequent itemsets are generated, association rules can be derived from them. These association rules show how often one item or group of items occurs when another item is present. This can be useful in the real world as it can reveal hidden relationships between things, such as the tendency to buy beer when also buying diapers.

In [ ]:
from itertools import chain, combinations

def AssociationRules(frequentItemSets, inputTextToList, minConfidence):
    associationRules = []
    for item in frequentItemSets:
        powerset = list(chain.from_iterable(combinations(item, r) for r in range(1,len(item))))
        for set in powerset:
            confidence = (Support(item, inputTextToList) / Support(list(set), inputTextToList))
            if confidence >= minConfidence:
                associationRules.append([list(set),[x for x in item if x not in list(set)], confidence])
    return associationRules

To generate association rules, every itemset in the frequent itemsets must be broken down into a power set or a list of every possible combination of items in the itemset. With this power set, the confidence can be calculated using the support method from above. To do this, the support for the itemset the power set was generated from is divided by the support for each item in the power set. If the confidence for that item in the power set is greater than or equal to the minimum confidence, then it is added to the list of association rules.